In [7]:
from github import Github, GithubException, Repository, ContentFile
import re
from typing import List

In [8]:
#Remove access key before commiting to GitHub
github = Github("dummy")

In [9]:
mtd_team_repos = github.get_organization("hmrc").get_team(1996975).get_repos()

In [32]:
class Program:
    @staticmethod
    def getMicroserviceVersions(repo: Repository):
        version_regex = "app/v[\d]+"
        try:
            app_folder_contents = repo.get_contents("app")
            microservice_versions = []  
            for item in app_folder_contents:
                if re.match(version_regex, item.path) is not None:
                    microservice_versions.append(item.path[4:])
            return microservice_versions
        except Exception as ex:
            raise ex
    
    @staticmethod
    def getErrorSummaries(mtd_team_repos: List[Repository.Repository]):
        error_summaries = []
        for mtd_team_repo in mtd_team_repos:
            try:
                mtd_team_repo.get_contents("app")
                microservice_versions = Program.getMicroserviceVersions(mtd_team_repo)
                for version in microservice_versions:
                    error_summaries.append(ErrorSummary(mtd_team_repo, version))
            except GithubException as ex:
                if not(ex.status == 404):
                    raise ex
            except Exception as ex:
                raise ex
        return error_summaries
              
    def __init__(self, repos: List[Repository.Repository]):
        self.data = Program.getErrorSummaries(repos)

In [33]:
class ErrorSummary:
    @staticmethod
    def getErrorContent(raw_error_content: ContentFile):
        boilerplate_regex = "\/\*\n(.*\*.*\n)*.*\*\/|.*\/\/.*\n|package.*\n|import.*\n"
        decoded_error_content = raw_error_content.decoded_content.decode("utf-8")
        error_content_without_boilerplate = re.sub(boilerplate_regex, "", decoded_error_content)
        return error_content_without_boilerplate
    
    @staticmethod
    def hasPaths(repo: Repository, microservice_version):
        error_class_regex = "case.*class.*Error\(.*code.*\,.*?.*message.*\)"
        try:
            microservice_version_error_files = repo.get_contents("app/" + microservice_version + "/models/errors")
            error_class_files = []
            for file in microservice_version_error_files:
                error_class_matches = re.search(error_class_regex, file.decoded_content.decode("utf-8"))
                if error_class_matches is not None:
                    error_class_files.append(file)
            if (len(error_class_files) == 1):
                aa = error_class_files[0]
            elif error_class_files == []:
                raise RuntimeError("[MicroserviceErrorSummary][getMicroserviceVersionErrorSummary] - No error case class could be found for " +
                                   "repository: " + repo.name +
                                   ", microservice version: " + microservice_version)
            else:
                raise RuntimeError("[MicroserviceErrorSummary][getMicroserviceVersionErrorSummary] - Multiple conflicting error case classes found for " +
                                   "repository: " + repo.name +
                                   ", microservice version: " + microservice_version)            
        except Exception as ex:
            raise ex
        
        reg = "(?<=case class MtdError\().*?(?=\))|(?<=case class Error\().*?(?=\))"
        ans = re.search(reg, ErrorSummary.getErrorContent(aa))
        if ans == None:
            raise RuntimeError("[ErrorVersion][hasPaths]")
        error_params = ans.group().split(",")
        for error_param in error_params:
            if "paths" in error_param:
                return True
        return False
    
    @staticmethod
    def getErrors(repo: Repository, microservice_version):
        error_regex = "object *[A-Za-z]* *extends *[A-Za-z]*Error\(.*?\)"
        aa = repo.get_contents("app/" + microservice_version + "/models/errors")
        for file in aa:
            raw_error_strings = re.findall(error_regex, ErrorSummary.getErrorContent(file), re.DOTALL)
            errors = []
            for raw_error_string in raw_error_strings:
                errors.append(Error(raw_error_string.replace("\n", "")))
            if errors != []:
                return errors
        raise RuntimeError("[ErrorVersion][getErrors]")
            
    def __init__(self, repo: Repository, microservice_version: str):
        self.repo_name = repo.name
        self.microservice_version = microservice_version
        self.has_paths = ErrorSummary.hasPaths(repo, microservice_version)
        self.errors = ErrorSummary.getErrors(repo, microservice_version)

In [34]:
class Error:
    @staticmethod
    def getErrorName(raw_error_string: str):
        error_name = re.search("(?<=object).*(?=.*extends.*Error\(.*?\))", raw_error_string)
        return error_name.group().replace(" ","")
    
    @staticmethod
    def getErrorCode(raw_error_string: str):
        error_code = re.search('(?<=( |\(|=|,)")[A-Z_]*(?=")', raw_error_string)
        return error_code.group()
    
    @staticmethod
    def getErrorMessage(raw_error_string: str):
        error_message = re.findall('"[^,]*?.*?"', raw_error_string)
        try:
            return error_message[1].replace('"',"")
        except Exception as ex:
            raise ex
    
    def __init__(self, raw_error_string: str):
        self.error_name = Error.getErrorName(raw_error_string)
        self.error_code = Error.getErrorCode(raw_error_string)
        self.error_message = Error.getErrorMessage(raw_error_string)  

In [35]:
ans = Program(mtd_team_repos)